In [2]:
print("""
FullName : Diallo Mamadou Korka
Studend’s Number: 022028845F
      
Modelling and Analysis of Complex Networks
Assignment : 9 
Number of the dataset : 3
""")


FullName : Diallo Mamadou Korka
Studend’s Number: 022028845F
      
Modelling and Analysis of Complex Networks
Assignment : 9 
Number of the dataset : 3



In [18]:
# import the modules
import urllib.request
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, roc_curve
import torch
import torch.nn as nn
import torch.optim as optim


In [10]:
#Load the dataset
# Load the Facebook-Ego network
facebook_url = "https://raw.githubusercontent.com/wang422003/Complex-Networks_exercise/main/Datasets/Group3/Facebook-Ego/348.edges"
urllib.request.urlretrieve(facebook_url, "facebook.edges")
facebook_network = nx.read_edgelist("facebook.edges", nodetype=int)

# Load the Twitter-Ego network
twitter_url = "https://raw.githubusercontent.com/wang422003/Complex-Networks_exercise/main/Datasets/Group3/Twitter-Ego/789071.edges"
urllib.request.urlretrieve(twitter_url, "twitter.edges")
twitter_network = nx.read_edgelist("twitter.edges", nodetype=int, create_using=nx.DiGraph())

In [11]:
# Convert the graph to an adjacency matrix
adjacency_matrix = nx.to_numpy_matrix(facebook_network)

In [12]:
# Generate random labels (0 or 1) for the edges
labels = np.random.choice([0, 1], size=(facebook_network.number_of_nodes(), facebook_network.number_of_nodes()))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(adjacency_matrix, labels, test_size=0.2, random_state=42)


In [15]:
# Define the neural network model
class LinkPredictionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LinkPredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [16]:
# Initialize the model
input_size = adjacency_matrix.shape[1]
hidden_size = 64
output_size = 1  # Binary classification

In [19]:
model = LinkPredictionModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100

In [24]:
from torch.utils.data import DataLoader, TensorDataset

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train.flatten())

# Create a TensorDataset
dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Create a DataLoader
batch_size = 32  # You can adjust this based on your dataset size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for batch_inputs, batch_labels in dataloader:
        # Forward pass
        outputs = model(batch_inputs)

        # Reshape the outputs to match the shape of the labels
        outputs = outputs.view(-1)

        # Calculate the loss
        loss = criterion(outputs, batch_labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


AssertionError: Size mismatch between tensors